## Overture

In [89]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from tqdm import tqdm

In [2]:
SERVICE_ACCOUNT_FILE = './config/local/rd-multicanal-caip-prod-0ec5b75f6a1b.json'

In [3]:
credentials = service_account.Credentials.from_service_account_file( SERVICE_ACCOUNT_FILE)

## Connection

In [4]:
client = bigquery.Client(credentials=credentials)

## Explore

### List Datasets

In [5]:
datasets = client.list_datasets()

print("Datasets in project '{}':".format(client.project))
for dataset in datasets:
    print(dataset.dataset_id)

Datasets in project 'rd-multicanal-caip-prod':
caip
caip2
kibana
orders


### List tables in dataset

In [6]:
tables = client.list_tables('caip')
print("\tTables contained in '{}':".format(dataset.dataset_id))
for table in tables:
    print("\t\t{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

	Tables contained in 'orders':
		rd-multicanal-caip-prod.caip.comment
		rd-multicanal-caip-prod.caip.feedback


### List columns in a table

In [7]:
dataset_id = 'caip'
table_id = 'comment'
table_ref = client.dataset(dataset_id).table(table_id)
table_obj = client.get_table(table_ref)
print("\t\t\tColumns in '{}':".format(table_id))
for field in table_obj.schema:
    print("\t\t\t\t'{}'".format(field.name))

			Columns in 'comment':
				'sk_comment'
				'sk_feedback'
				'comment'
				'created_ts'


## Exp0

### save dict of dataframes with column types

In [51]:
dtables = {}
datasets = client.list_datasets()
for dataset in datasets:
    tables = client.list_tables(dataset)
    for table in tables:
        table_id = f'{dataset.dataset_id}.{table.table_id}'
        dtables[table_id] = {}
        table_ref = client.dataset(dataset.dataset_id).table(table.table_id)
        table_obj = client.get_table(table_ref)
        for field in table_obj.schema:
            dtables[table_id][field.name] = field.field_type

In [52]:
dftables = {}
for table in dtables:
    dftables[table] = pd.DataFrame.from_dict(dtables[table], orient='index', columns=['type'])

### write each df to a excel sheet inside an excel file

In [53]:

writer = pd.ExcelWriter('rd-multicanal-caip-prod_column_types.xlsx',engine='xlsxwriter')   
for table_id, table in dftables.items():
    table.to_excel(writer, sheet_name=table_id)
writer.close()

***

### count rows in all tables

In [38]:
d_len_tables = {}
datasets = client.list_datasets()
for dataset in datasets:
    tables = client.list_tables(dataset)
    d_len_tables[dataset.dataset_id] = {}
    for table in tables:
        table_id = f'{dataset.dataset_id}.{table.table_id}'
        QUERY = (
            
        )
        d_len_tables[dataset.dataset_id][table.table_id] = client.query(f'SELECT count(*) FROM {table_id}').to_dataframe().values[0][0]  # API request

Datasets in project 'rd-multicanal-caip-prod':


In [39]:
d_len_tables['caip']['comment']

244

In [44]:
df_len_tables = {}
for dataset_id in d_len_tables.keys():
    df_len_tables[dataset_id] = pd.DataFrame.from_dict(d_len_tables[dataset_id], orient='index', columns=['n_rows'])

In [45]:
df_len_tables['caip']

,n_rows
comment,244
feedback,368


In [ ]:

writer = pd.ExcelWriter('rd-multicanal-caip-prod_nrows.xlsx',engine='xlsxwriter')   
for dataset_id, tables_count in df_len_tables.items():
    tables_count.to_excel(writer, sheet_name=dataset_id)
writer.close()

### Get all timestamp columns

In [60]:
time_cols = {}
datasets = client.list_datasets()
for dataset in datasets:
    tables = client.list_tables(dataset)
    for table in tables:
        table_id = f'{dataset.dataset_id}.{table.table_id}'
        time_cols[table_id] = []
        table_ref = client.dataset(dataset.dataset_id).table(table.table_id)
        table_obj = client.get_table(table_ref)
        for field in table_obj.schema:
            if field.field_type == 'TIMESTAMP':
                time_cols[table_id].append(field.name)

In [61]:
time_cols

{'caip.comment': ['created_ts'],
 'caip.feedback': ['created_ts'],
 'caip2.comment': ['created_ts'],
 'caip2.feedback': ['created_ts'],
 'caip2.feedback19JAN23': ['created_ts'],
 'kibana.call_logs': ['inicio_sessao_GMT_0',
  'fim_sessao_GMT_0',
  'inicio_sessao_GMT_BR',
  'fim_sessao_GMT_BR',
  'data_br',
  'created'],
 'kibana.call_logs_ETL_TESTE': ['inicio_sessao_GMT_0',
  'fim_sessao_GMT_0',
  'inicio_sessao_GMT_BR',
  'fim_sessao_GMT_BR',
  'data_br',
  'created'],
 'kibana.call_logs_bkp_01FEV23': ['inicio_sessao_GMT_0',
  'fim_sessao_GMT_0',
  'inicio_sessao_GMT_BR',
  'fim_sessao_GMT_BR',
  'data_br',
  'created'],
 'kibana.call_logs_bkp_13MAR23': ['inicio_sessao_GMT_0',
  'fim_sessao_GMT_0',
  'inicio_sessao_GMT_BR',
  'fim_sessao_GMT_BR',
  'data_br',
  'created'],
 'kibana.call_logs_bkp_19ABR23': ['inicio_sessao_GMT_0',
  'fim_sessao_GMT_0',
  'inicio_sessao_GMT_BR',
  'fim_sessao_GMT_BR',
  'data_br',
  'created'],
 'kibana.call_logs_debug': ['inicio_sessao_GMT_0',
  'fim_ses

### get date range for each timestamp columns as list of timestamp objects

In [93]:
list_time_ranges = {}
for table_id, table in tqdm(time_cols.items()):
    list_time_ranges[table_id] = {}
    for field_name in table:
        list_time_ranges[table_id][field_name] = (
            list(client.query(f'SELECT min({field_name}), max({field_name}) FROM {table_id}').to_dataframe().values[0]))

100%|██████████| 12/12 [01:05<00:00,  5.48s/it]


### convert timestamps to strings

In [ ]:
str_time_ranges = {}
for k,v in time_ranges.items():
    str_time_ranges[k] = {}
    for k1,v1 in v.items():
        str_time_ranges[k][k1] = [str(i) for i in v1]
#str_time_ranges = {k:i.strftime('%Y-%m-%d %H:%M:%S') for i in time_ranges['caip.comment']['created_at']]

### get date range for each timestamp columns as dataframes

In [140]:
df_time_ranges = {}
for table_id, table in tqdm(time_cols.items()):
    df = pd.DataFrame()
    for field_name in table:
        df_field = client.query(f'SELECT min({field_name}) as min_date, max({field_name}) as max_date FROM {table_id}').to_dataframe()
        df_field.index = [field_name]
        df = pd.concat((df, df_field))
    df_time_ranges[table_id]= df 
        

100%|██████████| 12/12 [00:45<00:00,  3.81s/it]


### Convert timestamp datezone to string

In [155]:
for k,v in df_time_ranges.items():
    #print(k, v)
    if len(v)>0:
        try:
            df_time_ranges[k]['min_date'] = v['min_date'].apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S'))
            df_time_ranges[k]['max_date'] = v['max_date'].apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S'))
        except Exception as e:
            print(e)
            

'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'
'str' object has no attribute 'strftime'


In [ ]:
df['caip.comment']

In [158]:
writer = pd.ExcelWriter('rd-multicanal-caip-prod_dates.xlsx',engine='xlsxwriter')   
for table_id, table_dates in df_time_ranges.items():
    print(table_id)
    table_dates.to_excel(writer, sheet_name=table_id)
writer.close()

caip.comment


ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

***

### Query

### Make query

In [9]:
QUERY = (
    'SELECT * FROM `caip.comment`'
    'LIMIT 100')
query_job = client.query(QUERY)  # API request

### get dataframe

In [10]:
df = query_job.to_dataframe()
df.head()

,sk_comment,sk_feedback,comment,created_ts
0,f6641b93-bbad-4346-b740-a7851b388a47,7e6da096-133e-4797-a3ba-c732d9164652,.,2022-07-27 23:12:24.317000+00:00
1,39dcfeb1-5dc6-4657-aaa7-98da0ec8571c,f0127695-3fde-499a-8d1c-a78b09e70d6c,4,2022-06-08 15:26:17.861000+00:00
2,95609881-7521-4171-9f9c-ae754dfc3db4,f8b8a295-298a-4e01-ae16-bb4e266fea80,A,2022-07-28 15:51:06.667000+00:00
3,a1b46379-9d67-4b39-ba68-0bde208b77ff,46581203-f334-48ae-974b-6a0c530fe075,Oi,2022-07-26 02:14:08.516000+00:00
4,dc66e0a5-5c5f-4426-8088-c6661d1b065b,14c0cb20-5df8-42e7-8834-159e7a253599,oii,2022-07-28 18:44:24.551000+00:00


### get rows from query result

In [11]:
rows = query_job.result()
for row in rows:
    print(row)

Row(('f6641b93-bbad-4346-b740-a7851b388a47', '7e6da096-133e-4797-a3ba-c732d9164652', '.', datetime.datetime(2022, 7, 27, 23, 12, 24, 317000, tzinfo=datetime.timezone.utc)), {'sk_comment': 0, 'sk_feedback': 1, 'comment': 2, 'created_ts': 3})
Row(('39dcfeb1-5dc6-4657-aaa7-98da0ec8571c', 'f0127695-3fde-499a-8d1c-a78b09e70d6c', '4', datetime.datetime(2022, 6, 8, 15, 26, 17, 861000, tzinfo=datetime.timezone.utc)), {'sk_comment': 0, 'sk_feedback': 1, 'comment': 2, 'created_ts': 3})
Row(('95609881-7521-4171-9f9c-ae754dfc3db4', 'f8b8a295-298a-4e01-ae16-bb4e266fea80', 'A', datetime.datetime(2022, 7, 28, 15, 51, 6, 667000, tzinfo=datetime.timezone.utc)), {'sk_comment': 0, 'sk_feedback': 1, 'comment': 2, 'created_ts': 3})
Row(('a1b46379-9d67-4b39-ba68-0bde208b77ff', '46581203-f334-48ae-974b-6a0c530fe075', 'Oi', datetime.datetime(2022, 7, 26, 2, 14, 8, 516000, tzinfo=datetime.timezone.utc)), {'sk_comment': 0, 'sk_feedback': 1, 'comment': 2, 'created_ts': 3})
Row(('dc66e0a5-5c5f-4426-8088-c6661d1b0